In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import sys
sys.path.append('utils/')

import keras
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# use non standard flow_from_directory
from image_preprocessing_ver2 import ImageDataGenerator
# it outputs y_batch that contains onehot targets and logits
# logits came from xception
from tqdm import tqdm
from keras.models import Model
from keras.layers import Lambda, concatenate, Activation,Dense
from keras.losses import categorical_crossentropy as logloss
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy
from keras import backend as K
from keras.losses import mean_squared_error as l2
from keras.losses import mean_absolute_error as l1
from mobilenet import get_mobilenet
from keras.applications.mobilenet import preprocess_input
import pandas as pd
import tensorflow as tf
#from  tensorflow.losses import absolute_difference as l1
#from  tensorflow.losses import mean_squared_error as l2


Using TensorFlow backend.


In [3]:
data_dir = './'

In [4]:
train_logits = np.load(data_dir + 'datatrain_logits.npy')[()]
val_logits = np.load(data_dir + 'dataval_logits.npy')[()]
print(train_logits[list(train_logits.keys())[0]].shape)

(8,)


In [5]:
data_generator = ImageDataGenerator(
    data_format='channels_last',
    preprocessing_function=preprocess_input
)

# note: i'm also passing dicts of logits
train_generator = data_generator.flow_from_directory(
    data_dir + 'data/train', train_logits,
    target_size=(224, 224),
    batch_size=16
)

val_generator = data_generator.flow_from_directory(
    data_dir + 'data/val', val_logits,
    target_size=(224, 224),
    batch_size=16
)

Found 4383 images belonging to 1 classes.
Found 399 images belonging to 1 classes.


# Create model

In [6]:
model = get_mobilenet(224, alpha=0.25, weight_decay=1e-5, dropout=0.1)

# remove softmax
model.layers.pop()

# usual probabilities
logits = model.layers[-1].output
probabilities = Activation('softmax')(logits)

boxes = Dense(4)(probabilities)
reshaped = keras.layers.Reshape((4,))(boxes)

model = Model(model.input, reshaped)

def accuracy(y_true,y_pred):
    return

def bounded_loss(rt, rs, m, v):
    """
        Input: Rs:     regression output from student network
               Rt:     regression output from teacher
               y_true: ground truth bounding box

        Output:Loss
    """
    #if(l2(rs,rt[1]) + m > l2(rt[0] , rt[1]) ):
        #lb = l2(rs,rt[1])
    #else:
        #lb = tf.constant(0,dtype=float)
    #lreg =l1(rs,rt[1]) + v*lb
    print(rt)
    #print(rs)
    #print(x)
    bound = l2(rs,rt[:, 4:]) + K.constant(m,shape=(1,))
    bound2 = l2(rt[:, :4] , rt[:, 4:])
    cond = tf.less(tf.reduce_mean(bound), tf.reduce_mean(bound2))
    #print(bound2.shape)
    
    #lb = tf.Variable(0.0, name="lb")
    #lreg = tf.Variable(0.0, name="lreg")
    #lreg = K.mean(K.square(rs - x[1]))
    #tf.cond(l2(rs,x[1]) + m > l2(x[0] , x[1]),lambda:tf.assign(lb,l2(rs,x[1])),lambda:tf.assign(lb,0.0))
    loss2 = lambda: l2(rs,rt[:, 4:])
    zero = lambda: K.constant(0.0,shape=(1,))
    res = tf.cond(cond,loss2,zero)
    return res

def bounded_loss_nogt(rs, rt, m, v):
    """        when the dataset is not available
        Input: Rs:     regression output from student network
               Rt:     regression output from teacher

        Output:Loss
    """

    #tf.cond(((l2(rs,rt) < (l2(rs,rt) + m ))or (l2(rs,rt) > (l2(rs,rt) - m))),lambda:tf.assign(lb,0),lambda:tf.assign(lb,l2(rs,rt)))

    #if((l2(rs,rt) < (l2(rs,rt) + m ))or (l2(rs,rt) > (l2(rs,rt) - m))):
        #lb = 0
    #else:
        #lb = l2(rs,rt)
        
    
    return 5
def toyLoss(y_true, y_pred):
    print(y_true[1][3])
    print(y_pred)
    return tf.multiply(3.9,4.6)

In [7]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

#Custom Accuracy for IoU in keras 
def IoU_Dataset(y_true, y_pred):

    
    xA = tf.maximum(y_pred[:,0],y_true[:,4])
    yA = tf.maximum(y_pred[:,1],y_true[:,5])
    xB = tf.maximum(y_pred[:,2],y_true[:,6])
    yB = tf.maximum(y_pred[:,3],y_true[:,7])
    
    interArea = tf.maximum(K.constant(0.0,shape=(1,)), xB - xA + 1) * tf.maximum(K.constant(0.0,shape=(1,)), yB - yA + 1)
    boxAArea = (y_pred[:,2] - y_pred[:,0] + 1) * (y_pred[:,3] - y_pred[:,1] + 1)
    boxBArea = (y_true[:,6] - y_true[:,4] + 1) * (y_true[:,7] - y_true[:,5] + 1)
    
    iou = interArea / (boxAArea + boxBArea - interArea)
    return iou
    
def IoU_Teacher(y_true, y_pred):
    tf.cast(
    y_true,
    tf.float32,
    name=None)
    
    tf.cast(
    y_pred,
    tf.float32,
    name=None)
    
    xA = tf.maximum(y_pred[:,0],y_true[:,0])
    yA = tf.maximum(y_pred[:,1],y_true[:,1])
    xB = tf.maximum(y_pred[:,2],y_true[:,2])
    yB = tf.maximum(y_pred[:,3],y_true[:,3])
    
    interArea = tf.maximum(K.constant(0.0,shape=(1,)), xB - xA + 1) * tf.maximum(K.constant(0.0,shape=(1,)), yB - yA + 1)
    boxAArea = (y_pred[:,2] - y_pred[:,0] + 1) * (y_pred[:,3] - y_pred[:,1] + 1)
    boxBArea = (y_true[:,2] - y_true[:,0] + 1) * (y_true[:,3] - y_true[:,1] + 1)
    iou = interArea / (boxAArea + boxBArea - interArea)
    return iou

In [8]:
print(model.layers[-1].output_shape)
model.compile(
    optimizer=optimizers.SGD(lr=1e-1, momentum=0.9, nesterov=True), 
    loss=lambda y_true, y_pred:bounded_loss(y_true,y_pred,0.5,0.5),metrics=[IoU_Teacher,IoU_Dataset]
    
)

model.fit_generator(
    train_generator, 
    steps_per_epoch=250, epochs=50, verbose=1,
    validation_data=val_generator, validation_steps=80, workers=4
)

(None, 4)
Tensor("reshape_1_target:0", shape=(?, ?), dtype=float32)
Epoch 1/50


116/250 [============>.................] - ETA: 22:46 - loss: 0.0026 - IoU_Teacher: 3.3323 - IoU_Dataset: 1.46 - ETA: 14:17 - loss: 0.0026 - IoU_Teacher: 3.3334 - IoU_Dataset: 1.44 - ETA: 11:23 - loss: 0.0026 - IoU_Teacher: 3.3338 - IoU_Dataset: 1.46 - ETA: 9:57 - loss: 0.0026 - IoU_Teacher: 3.3341 - IoU_Dataset: 1.4566 - ETA: 9:06 - loss: 0.0026 - IoU_Teacher: 3.3342 - IoU_Dataset: 1.453 - ETA: 8:31 - loss: 0.0026 - IoU_Teacher: 3.3345 - IoU_Dataset: 1.448 - ETA: 8:05 - loss: 0.0026 - IoU_Teacher: 3.3343 - IoU_Dataset: 1.452 - ETA: 7:45 - loss: 0.0026 - IoU_Teacher: 3.3345 - IoU_Dataset: 1.451 - ETA: 7:29 - loss: 0.0026 - IoU_Teacher: 3.3345 - IoU_Dataset: 1.442 - ETA: 7:18 - loss: 0.0026 - IoU_Teacher: 3.3344 - IoU_Dataset: 1.444 - ETA: 7:14 - loss: 0.0026 - IoU_Teacher: 3.3345 - IoU_Dataset: 1.439 - ETA: 7:16 - loss: 0.0026 - IoU_Teacher: 3.3345 - IoU_Dataset: 1.442 - ETA: 7:16 - loss: 0.0026 - IoU_Teacher: 3.3346 - IoU_Dataset: 1.437 - ETA: 7:19 - loss: 0.0026 - IoU_Teacher: 3.3347

250/250 [==============================] - ETA: 28s - loss: 0.0026 - IoU_Teacher: 3.3347 - IoU_Dataset: 1.43 - ETA: 26s - loss: 0.0026 - IoU_Teacher: 3.3347 - IoU_Dataset: 1.42 - ETA: 24s - loss: 0.0026 - IoU_Teacher: 3.3347 - IoU_Dataset: 1.42 - ETA: 22s - loss: 0.0026 - IoU_Teacher: 3.3347 - IoU_Dataset: 1.42 - ETA: 21s - loss: 0.0026 - IoU_Teacher: 3.3347 - IoU_Dataset: 1.42 - ETA: 19s - loss: 0.0026 - IoU_Teacher: 3.3347 - IoU_Dataset: 1.42 - ETA: 17s - loss: 0.0026 - IoU_Teacher: 3.3346 - IoU_Dataset: 1.42 - ETA: 15s - loss: 0.0026 - IoU_Teacher: 3.3346 - IoU_Dataset: 1.42 - ETA: 13s - loss: 0.0026 - IoU_Teacher: 3.3346 - IoU_Dataset: 1.42 - ETA: 12s - loss: 0.0026 - IoU_Teacher: 3.3346 - IoU_Dataset: 1.42 - ETA: 10s - loss: 0.0026 - IoU_Teacher: 3.3346 - IoU_Dataset: 1.42 - ETA: 8s - loss: 0.0026 - IoU_Teacher: 3.3346 - IoU_Dataset: 1.4294 - ETA: 6s - loss: 0.0026 - IoU_Teacher: 3.3346 - IoU_Dataset: 1.429 - ETA: 5s - loss: 0.0026 - IoU_Teacher: 3.3346 - IoU_Dataset: 1.429 - ETA:

117/250 [=============>................] - ETA: 7:45 - loss: 0.0026 - IoU_Teacher: 3.3344 - IoU_Dataset: 1.446 - ETA: 7:16 - loss: 0.0026 - IoU_Teacher: 3.3344 - IoU_Dataset: 1.401 - ETA: 7:11 - loss: 0.0026 - IoU_Teacher: 3.3346 - IoU_Dataset: 1.414 - ETA: 6:58 - loss: 0.0026 - IoU_Teacher: 3.3350 - IoU_Dataset: 1.416 - ETA: 6:47 - loss: 0.0026 - IoU_Teacher: 3.3344 - IoU_Dataset: 1.442 - ETA: 6:41 - loss: 0.0026 - IoU_Teacher: 3.3343 - IoU_Dataset: 1.443 - ETA: 6:36 - loss: 0.0026 - IoU_Teacher: 3.3343 - IoU_Dataset: 1.450 - ETA: 6:30 - loss: 0.0026 - IoU_Teacher: 3.3342 - IoU_Dataset: 1.445 - ETA: 6:26 - loss: 0.0026 - IoU_Teacher: 3.3343 - IoU_Dataset: 1.440 - ETA: 6:23 - loss: 0.0026 - IoU_Teacher: 3.3342 - IoU_Dataset: 1.439 - ETA: 6:19 - loss: 0.0026 - IoU_Teacher: 3.3341 - IoU_Dataset: 1.433 - ETA: 6:15 - loss: 0.0026 - IoU_Teacher: 3.3340 - IoU_Dataset: 1.424 - ETA: 6:12 - loss: 0.0026 - IoU_Teacher: 3.3343 - IoU_Dataset: 1.433 - ETA: 6:09 - loss: 0.0026 - IoU_Teacher: 3.3344 

250/250 [==============================] - ETA: 21s - loss: 0.0025 - IoU_Teacher: 3.3345 - IoU_Dataset: 1.42 - ETA: 20s - loss: 0.0025 - IoU_Teacher: 3.3345 - IoU_Dataset: 1.42 - ETA: 18s - loss: 0.0025 - IoU_Teacher: 3.3345 - IoU_Dataset: 1.42 - ETA: 17s - loss: 0.0025 - IoU_Teacher: 3.3345 - IoU_Dataset: 1.42 - ETA: 15s - loss: 0.0025 - IoU_Teacher: 3.3345 - IoU_Dataset: 1.42 - ETA: 14s - loss: 0.0025 - IoU_Teacher: 3.3345 - IoU_Dataset: 1.42 - ETA: 12s - loss: 0.0025 - IoU_Teacher: 3.3345 - IoU_Dataset: 1.42 - ETA: 10s - loss: 0.0025 - IoU_Teacher: 3.3345 - IoU_Dataset: 1.42 - ETA: 9s - loss: 0.0025 - IoU_Teacher: 3.3345 - IoU_Dataset: 1.4287 - ETA: 7s - loss: 0.0025 - IoU_Teacher: 3.3345 - IoU_Dataset: 1.428 - ETA: 6s - loss: 0.0025 - IoU_Teacher: 3.3345 - IoU_Dataset: 1.428 - ETA: 4s - loss: 0.0025 - IoU_Teacher: 3.3345 - IoU_Dataset: 1.428 - ETA: 3s - loss: 0.0025 - IoU_Teacher: 3.3345 - IoU_Dataset: 1.428 - ETA: 1s - loss: 0.0025 - IoU_Teacher: 3.3345 - IoU_Dataset: 1.428 - 572s

...

250/250 [==============================] - ETA: 21s - loss: 0.0016 - IoU_Teacher: 3.3310 - IoU_Dataset: 1.42 - ETA: 19s - loss: 0.0016 - IoU_Teacher: 3.3310 - IoU_Dataset: 1.42 - ETA: 18s - loss: 0.0016 - IoU_Teacher: 3.3310 - IoU_Dataset: 1.42 - ETA: 16s - loss: 0.0016 - IoU_Teacher: 3.3310 - IoU_Dataset: 1.42 - ETA: 15s - loss: 0.0016 - IoU_Teacher: 3.3310 - IoU_Dataset: 1.42 - ETA: 13s - loss: 0.0016 - IoU_Teacher: 3.3310 - IoU_Dataset: 1.42 - ETA: 12s - loss: 0.0016 - IoU_Teacher: 3.3310 - IoU_Dataset: 1.42 - ETA: 10s - loss: 0.0016 - IoU_Teacher: 3.3310 - IoU_Dataset: 1.42 - ETA: 9s - loss: 0.0016 - IoU_Teacher: 3.3310 - IoU_Dataset: 1.4276 - ETA: 7s - loss: 0.0016 - IoU_Teacher: 3.3310 - IoU_Dataset: 1.427 - ETA: 6s - loss: 0.0016 - IoU_Teacher: 3.3310 - IoU_Dataset: 1.428 - ETA: 4s - loss: 0.0016 - IoU_Teacher: 3.3310 - IoU_Dataset: 1.427 - ETA: 3s - loss: 0.0016 - IoU_Teacher: 3.3310 - IoU_Dataset: 1.427 - ETA: 1s - loss: 0.0016 - IoU_Teacher: 3.3310 - IoU_Dataset: 1.427 - 567s

117/250 [=============>................] - ETA: 7:24 - loss: 0.0016 - IoU_Teacher: 3.3309 - IoU_Dataset: 1.434 - ETA: 6:47 - loss: 0.0016 - IoU_Teacher: 3.3308 - IoU_Dataset: 1.460 - ETA: 6:33 - loss: 0.0016 - IoU_Teacher: 3.3308 - IoU_Dataset: 1.448 - ETA: 6:23 - loss: 0.0016 - IoU_Teacher: 3.3308 - IoU_Dataset: 1.449 - ETA: 6:16 - loss: 0.0016 - IoU_Teacher: 3.3308 - IoU_Dataset: 1.439 - ETA: 6:10 - loss: 0.0016 - IoU_Teacher: 3.3308 - IoU_Dataset: 1.450 - ETA: 6:06 - loss: 0.0016 - IoU_Teacher: 3.3309 - IoU_Dataset: 1.448 - ETA: 6:03 - loss: 0.0016 - IoU_Teacher: 3.3307 - IoU_Dataset: 1.441 - ETA: 6:00 - loss: 0.0016 - IoU_Teacher: 3.3307 - IoU_Dataset: 1.435 - ETA: 5:57 - loss: 0.0016 - IoU_Teacher: 3.3307 - IoU_Dataset: 1.431 - ETA: 5:55 - loss: 0.0016 - IoU_Teacher: 3.3306 - IoU_Dataset: 1.435 - ETA: 5:52 - loss: 0.0016 - IoU_Teacher: 3.3306 - IoU_Dataset: 1.432 - ETA: 5:50 - loss: 0.0016 - IoU_Teacher: 3.3307 - IoU_Dataset: 1.431 - ETA: 5:48 - loss: 0.0016 - IoU_Teacher: 3.3308 

250/250 [==============================] - ETA: 21s - loss: 0.0016 - IoU_Teacher: 3.3309 - IoU_Dataset: 1.42 - ETA: 20s - loss: 0.0016 - IoU_Teacher: 3.3309 - IoU_Dataset: 1.42 - ETA: 18s - loss: 0.0016 - IoU_Teacher: 3.3309 - IoU_Dataset: 1.42 - ETA: 16s - loss: 0.0016 - IoU_Teacher: 3.3309 - IoU_Dataset: 1.42 - ETA: 15s - loss: 0.0016 - IoU_Teacher: 3.3309 - IoU_Dataset: 1.42 - ETA: 13s - loss: 0.0016 - IoU_Teacher: 3.3309 - IoU_Dataset: 1.42 - ETA: 12s - loss: 0.0016 - IoU_Teacher: 3.3309 - IoU_Dataset: 1.42 - ETA: 10s - loss: 0.0016 - IoU_Teacher: 3.3309 - IoU_Dataset: 1.42 - ETA: 9s - loss: 0.0016 - IoU_Teacher: 3.3309 - IoU_Dataset: 1.4258 - ETA: 7s - loss: 0.0016 - IoU_Teacher: 3.3309 - IoU_Dataset: 1.426 - ETA: 6s - loss: 0.0016 - IoU_Teacher: 3.3309 - IoU_Dataset: 1.426 - ETA: 4s - loss: 0.0016 - IoU_Teacher: 3.3309 - IoU_Dataset: 1.425 - ETA: 3s - loss: 0.0016 - IoU_Teacher: 3.3309 - IoU_Dataset: 1.425 - ETA: 1s - loss: 0.0016 - IoU_Teacher: 3.3309 - IoU_Dataset: 1.425 - 567s

117/250 [=============>................] - ETA: 6:10 - loss: 0.0016 - IoU_Teacher: 3.3291 - IoU_Dataset: 1.408 - ETA: 6:08 - loss: 0.0016 - IoU_Teacher: 3.3292 - IoU_Dataset: 1.430 - ETA: 6:03 - loss: 0.0016 - IoU_Teacher: 3.3296 - IoU_Dataset: 1.433 - ETA: 5:59 - loss: 0.0016 - IoU_Teacher: 3.3302 - IoU_Dataset: 1.433 - ETA: 6:08 - loss: 0.0016 - IoU_Teacher: 3.3303 - IoU_Dataset: 1.424 - ETA: 6:08 - loss: 0.0016 - IoU_Teacher: 3.3305 - IoU_Dataset: 1.427 - ETA: 6:05 - loss: 0.0016 - IoU_Teacher: 3.3306 - IoU_Dataset: 1.429 - ETA: 6:02 - loss: 0.0016 - IoU_Teacher: 3.3305 - IoU_Dataset: 1.423 - ETA: 6:00 - loss: 0.0016 - IoU_Teacher: 3.3305 - IoU_Dataset: 1.423 - ETA: 5:59 - loss: 0.0016 - IoU_Teacher: 3.3305 - IoU_Dataset: 1.423 - ETA: 5:56 - loss: 0.0016 - IoU_Teacher: 3.3305 - IoU_Dataset: 1.426 - ETA: 5:54 - loss: 0.0016 - IoU_Teacher: 3.3304 - IoU_Dataset: 1.430 - ETA: 5:52 - loss: 0.0016 - IoU_Teacher: 3.3306 - IoU_Dataset: 1.426 - ETA: 5:49 - loss: 0.0016 - IoU_Teacher: 3.3306 

250/250 [==============================] - ETA: 21s - loss: 0.0016 - IoU_Teacher: 3.3308 - IoU_Dataset: 1.42 - ETA: 19s - loss: 0.0016 - IoU_Teacher: 3.3308 - IoU_Dataset: 1.42 - ETA: 18s - loss: 0.0016 - IoU_Teacher: 3.3308 - IoU_Dataset: 1.42 - ETA: 16s - loss: 0.0016 - IoU_Teacher: 3.3308 - IoU_Dataset: 1.42 - ETA: 15s - loss: 0.0016 - IoU_Teacher: 3.3307 - IoU_Dataset: 1.42 - ETA: 13s - loss: 0.0016 - IoU_Teacher: 3.3308 - IoU_Dataset: 1.42 - ETA: 12s - loss: 0.0016 - IoU_Teacher: 3.3308 - IoU_Dataset: 1.42 - ETA: 10s - loss: 0.0016 - IoU_Teacher: 3.3308 - IoU_Dataset: 1.42 - ETA: 9s - loss: 0.0016 - IoU_Teacher: 3.3307 - IoU_Dataset: 1.4276 - ETA: 7s - loss: 0.0016 - IoU_Teacher: 3.3307 - IoU_Dataset: 1.427 - ETA: 6s - loss: 0.0016 - IoU_Teacher: 3.3308 - IoU_Dataset: 1.427 - ETA: 4s - loss: 0.0016 - IoU_Teacher: 3.3307 - IoU_Dataset: 1.427 - ETA: 3s - loss: 0.0016 - IoU_Teacher: 3.3308 - IoU_Dataset: 1.427 - ETA: 1s - loss: 0.0016 - IoU_Teacher: 3.3308 - IoU_Dataset: 1.427 - 563s

In [9]:
model.save("model")